In [32]:
# Импортируем необходимые пакеты
import numpy as np

# Зададим обучающую выборку
m = 17
h = 0.7
# Попробовать h=0.1, h=0.7
eps = 0.00001
epsfunc = 0.06
dataX = [
    [0.1, 0.2, 0.3, 0.4],
    [0.2, 0.3, 0.4, 0.5],
    [0.3, 0.4, 0.5, 0.6],
    [0.4, 0.5, 0.6, 0.7],
    [0.5, 0.6, 0.7, 0.8],
    [0.6, 0.7, 0.8, 0.9],
    [0.7, 0.8, 0.9, 0],
    [0.8, 0.9, 0, -0.1],
    [0.9, 0, -0.1, -0.2],
    [0, -0.1, -0.2, -0.3],
    [-0.1, -0.2, -0.3, -0.4],
    [-0.2, -0.3, -0.4, -0.5],
    [-0.3, -0.4, -0.5, -0.6],
    [-0.4, -0.5, -0.6, -0.7],
    [-0.5, -0.6, -0.7, -0.8],
    [-0.6, -0.7, -0.8, -0.9],
    [-0.7, -0.8, -0.9, 0]
]
dataY = [1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 0, -1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7]
print('Задаем обучающую выборку')
print('Функция сигмоида 1/(1+exp(-x))')


def sigmoid(x):
    sig = 1 / (1 + np.exp(-x))
    return sig


def diffsig(x):
    diffsig = np.exp(x) / (1 + np.exp(x)) / (1 + np.exp(x))
    return diffsig


# Наша нейронная сеть будет иметь 2 нейрона во входном слое, 2 в скрытом и 1 в выходном
# Определим модель нейрона с 4 входами
def neuro_in(w1, w2, w3, w4, x1, x2, x3, x4):
    neu = sigmoid(w1 * x1 + w2 * x2 + w3 * x3 + w4 * x4)
    return neu


def neuro_deep(w1, w2, x1, x2):
    neu = sigmoid(w1 * x1 + w2 * x2)
    return neu


def neuro_out(w1, w2, x1, x2):
    neu = w1 * x1 + w2 * x2
    return neu


def diff_neuro_in(w1, w2, w3, w4, x1, x2, x3, x4):
    neu = diffsig(w1 * x1 + w2 * x2 + w3 * x3 + w4 * x4)
    return neu


def diff_neuro_deep(w1, w2, x1, x2):
    neu = diffsig(w1 * x1 + w2 * x2)
    return neu


# 0 этап алгоритма. Инициализируем веса сети.
# Веса нейронов входного слоя
w111 = 0.01
w112 = 0.02
w113 = 0.03
w114 = 0.04

w121 = 0.05
w122 = 0.06
w123 = 0.07
w124 = 0.08

# Веса нейронов скрытого слоя
w211 = -0.01
w212 = -0.02

w221 = -0.03
w222 = -0.04

# Веса нейрона выходного слоя
w311 = -0.05
w312 = -0.06


# Посчитаем функцию ошибки
def loss_func(w111, w112, w113, w114, w121, w122, w123, w124,
              w211, w212, w221, w222, w311, w312, dataY, dataX, m=17):
    E = 0
    for i in range(m):
        # Значения первого слоя
        neu11 = neuro_in(w111, w112, w113, w114, dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3])
        neu12 = neuro_in(w121, w122, w123, w124, dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3])

        # Значения второго слоя
        neu21 = neuro_deep(w211, w212, neu11, neu12)
        neu22 = neuro_deep(w221, w222, neu11, neu12)

        # Значение выхода сети
        neu31 = neuro_out(w311, w312, neu21, neu22)

        E += (dataY[i] - neu31) * (dataY[i] - neu31)
    return E / m


# Вычисляем значение функции потерь
loss_new = loss_func(w111, w112, w113, w114, w121, w122, w123, w124,
                     w211, w212, w221, w222, w311, w312, dataY, dataX)
print('1 этап алгоритма. Вычисляем значение функции потерь. E(w111,...,w312) = ',
      round(loss_new, 6))

# Посчитаем значения на выходах нейронов для 0 элемента обучающей выборки
# Входного слоя
neu_in = [0, 0]
neu_in[0] = neuro_in(w111, w112, w113, w114, dataX[0][0], dataX[0][1], dataX[0][2], dataX[0][3])
neu_in[1] = neuro_in(w121, w122, w123, w124, dataX[0][0], dataX[0][1], dataX[0][2], dataX[0][3])

# Скрытого слоя
neu_deep = [0, 0]
neu_deep[0] = neuro_deep(w211, w212, neu_in[0], neu_in[1])
neu_deep[1] = neuro_deep(w221, w222, neu_in[0], neu_in[1])

# Выходного слоя
neu_out = neuro_out(w311, w312, neu_deep[0], neu_deep[1])

# Рассчитаем ошибку ответа сети
delta = dataY[0] - neu_out

# Посчитаем значение градиента функции потерь, используя обратное распространение ошибки delta
# Производные для весов выходного слоя
diffEw311 = 2 * delta * neu_deep[0]
diffEw312 = 2 * delta * neu_deep[1]

# Производные для весов скрытого слоя
diffEw211 = 2 * delta * w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * neu_in[0]
diffEw212 = 2 * delta * w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * neu_in[1]

diffEw221 = 2 * delta * w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * neu_in[0]
diffEw222 = 2 * delta * w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * neu_in[1]

# Производные для весов входного слоя
diffEw111 = 2 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w211 +
                         w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w221
                         ) * diff_neuro_in(w111, w112, w113, w114,
                                           dataX[0][0], dataX[0][1], dataX[0][2], dataX[0][3]) * dataX[0][0]

diffEw112 = 2 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w211 +
                         w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w221
                         ) * diff_neuro_in(w111, w112, w113, w114,
                                           dataX[0][0], dataX[0][1], dataX[0][2], dataX[0][3]) * dataX[0][1]

diffEw113 = 2 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w211 +
                         w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w221
                         ) * diff_neuro_in(w111, w112, w113, w114,
                                           dataX[0][0], dataX[0][1], dataX[0][2], dataX[0][3]) * dataX[0][2]

diffEw114 = 2 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w211 +
                         w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w221
                         ) * diff_neuro_in(w111, w112, w113, w114,
                                           dataX[0][0], dataX[0][1], dataX[0][2], dataX[0][3]) * dataX[0][3]

diffEw121 = 2 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w212 +
                         w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w222
                         ) * diff_neuro_in(w121, w122, w123, w124,
                                           dataX[0][0], dataX[0][1], dataX[0][2], dataX[0][3]) * dataX[0][0]

diffEw122 = 2 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w212 +
                         w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w222
                         ) * diff_neuro_in(w121, w122, w123, w124,
                                           dataX[0][0], dataX[0][1], dataX[0][2], dataX[0][3]) * dataX[0][1]

diffEw123 = 2 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w212 +
                         w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w222
                         ) * diff_neuro_in(w121, w122, w123, w124,
                                           dataX[0][0], dataX[0][1], dataX[0][2], dataX[0][3]) * dataX[0][2]

diffEw124 = 2 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w212 +
                         w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w222
                         ) * diff_neuro_in(w121, w122, w123, w124,
                                           dataX[0][0], dataX[0][1], dataX[0][2], dataX[0][3]) * dataX[0][3]

# Вычисляем значение градиента функции потерь
print('2 этап алгоритма. Вычисляем значение градиента функции потерь. diffE(w111,...,w314) = (',
      round(diffEw111, 6),
      round(diffEw112, 6),
      round(diffEw113, 6),
      round(diffEw114, 6),

      round(diffEw121, 6),
      round(diffEw122, 6),
      round(diffEw123, 6),
      round(diffEw124, 6),

      round(diffEw211, 6),
      round(diffEw212, 6),
      round(diffEw221, 6),
      round(diffEw222, 6),

      round(diffEw311, 6),
      round(diffEw312, 6),
      ')')

# Корректируем веса
# Веса нейронов входного слоя
w111 = w111 + h * diffEw111
w112 = w112 + h * diffEw112
w113 = w113 + h * diffEw113
w114 = w114 + h * diffEw114

w121 = w121 + h * diffEw121
w122 = w122 + h * diffEw122
w123 = w123 + h * diffEw123
w124 = w124 + h * diffEw124

# Веса нейронов скрытого слоя
w211 = w211 + h * diffEw211
w212 = w212 + h * diffEw212

w221 = w221 + h * diffEw221
w222 = w222 + h * diffEw222

# Веса нейрона выходного слоя
w311 = w311 + h * diffEw311
w312 = w312 + h * diffEw312

print('3 этап алгоритма. Корректируем веса W = (',
      round(w111, 6),
      round(w112, 6),
      round(w113, 6),
      round(w114, 6),

      round(w121, 6),
      round(w122, 6),
      round(w123, 6),
      round(w124, 6),

      round(w211, 6),
      round(w212, 6),
      round(w221, 6),
      round(w222, 6),

      round(w311, 6),
      round(w312, 6),
      ')')

# Запускаем цикл ADAM
count_stop = 0
index = np.arange(17)
t = 0
# Задаем влияние инерции
gammav = 0.2
# Задаем влияние масштабирования
gammag = 0.2
# Инициализируем инерцию
v111 = 0.01
v112 = 0.01
v113 = 0.01
v114 = 0.01
v121 = 0.01
v122 = 0.01
v123 = 0.01
v124 = 0.01

v211 = 0.01
v212 = 0.01
v221 = 0.01
v222 = 0.01

v311 = 0.01
v312 = 0.01
# Инициализируем масштабирование
g111 = 0.02
g112 = 0.02
g113 = 0.02
g114 = 0.02
g121 = 0.02
g122 = 0.02
g123 = 0.02
g124 = 0.02

g211 = 0.02
g212 = 0.02
g221 = 0.02
g222 = 0.02

g311 = 0.02
g312 = 0.02
for k in range(4000):
    np.random.shuffle(index)
    loss_old = loss_new
    for i in index:
        # Посчитаем значения на выходах нейронов для 0 элемента обучающей выборки
        # Входного слоя
        neu_in = [0, 0]
        neu_in[0] = neuro_in(w111, w112, w113, w114, dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3])
        neu_in[1] = neuro_in(w121, w122, w123, w124, dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3])

        # Скрытого слоя
        neu_deep = [0, 0]
        neu_deep[0] = neuro_deep(w211, w212, neu_in[0], neu_in[1])
        neu_deep[1] = neuro_deep(w221, w222, neu_in[0], neu_in[1])

        # Выходного слоя
        neu_out = neuro_out(w311, w312, neu_deep[0], neu_deep[1])

        # Рассчитаем ошибку ответа сети
        delta = dataY[i] - neu_out

        # Посчитаем значение градиента функции потерь, используя обратное распространение ошибки delta
        # Производные для весов выходного слоя
        diffEw311 = 2 * delta * neu_deep[0]
        diffEw312 = 2 * delta * neu_deep[1]

        # Производные для весов скрытого слоя
        diffEw211 = 2.0 * delta * w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * neu_in[0]
        diffEw212 = 2.0 * delta * w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * neu_in[1]

        diffEw221 = 2.0 * delta * w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * neu_in[0]
        diffEw222 = 2.0 * delta * w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * neu_in[1]

        # Производные для весов входного слоя
        diffEw111 = 2.0 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w211 +
                                 w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w221
                                 ) * diff_neuro_in(w111, w112, w113, w114,
                                                   dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3]) * dataX[i][0]

        diffEw112 = 2.0 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w211 +
                                 w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w221
                                 ) * diff_neuro_in(w111, w112, w113, w114,
                                                   dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3]) * dataX[i][1]

        diffEw113 = 2.0 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w211 +
                                 w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w221
                                 ) * diff_neuro_in(w111, w112, w113, w114,
                                                   dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3]) * dataX[i][2]

        diffEw114 = 2.0 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w211 +
                                 w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w221
                                 ) * diff_neuro_in(w111, w112, w113, w114,
                                                   dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3]) * dataX[i][3]

        diffEw121 = 2.0 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w212 +
                                 w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w222
                                 ) * diff_neuro_in(w121, w122, w123, w124,
                                                   dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3]) * dataX[i][0]

        diffEw122 = 2.0 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w212 +
                                 w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w222
                                 ) * diff_neuro_in(w121, w122, w123, w124,
                                                   dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3]) * dataX[i][1]

        diffEw123 = 2.0 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w212 +
                                 w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w222
                                 ) * diff_neuro_in(w121, w122, w123, w124,
                                                   dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3]) * dataX[i][2]

        diffEw124 = 2.0 * delta * (w311 * diff_neuro_deep(w211, w212, neu_in[0], neu_in[1]) * w212 +
                                 w312 * diff_neuro_deep(w221, w222, neu_in[0], neu_in[1]) * w222
                                 ) * diff_neuro_in(w121, w122, w123, w124,
                                                   dataX[i][0], dataX[i][1], dataX[i][2], dataX[i][3]) * dataX[i][3]

        # Корректируем инерцияю
        gvt = (1 - gammav ** (t + 1))
        # Запишите формулу скользящего среднего для обновления инерции весов модели.
        v111 = (gammav*v111+(1-gammav)*diffEw111)/gvt
        v112 = (gammav*v112+(1-gammav)*diffEw112)/gvt
        v113 = (gammav*v113+(1-gammav)*diffEw113)/gvt
        v114 = (gammav*v114+(1-gammav)*diffEw114)/gvt
        v121 = (gammav*v121+(1-gammav)*diffEw121)/gvt
        v122 = (gammav*v122+(1-gammav)*diffEw122)/gvt
        v123 = (gammav*v123+(1-gammav)*diffEw123)/gvt
        v124 = (gammav*v124+(1-gammav)*diffEw124)/gvt

        v211 = (gammav*v211+(1-gammav)*diffEw211)/gvt
        v212 = (gammav*v212+(1-gammav)*diffEw212)/gvt
        v221 = (gammav*v221+(1-gammav)*diffEw221)/gvt
        v222 = (gammav*v222+(1-gammav)*diffEw222)/gvt

        v311 = (gammav*v311+(1-gammav)*diffEw311)/gvt
        v312 = (gammav*v312+(1-gammav)*diffEw312)/gvt

        # Корректируем масштабирование
        ggt = (1 - gammag ** (t + 1))
        # Запишите формулу скользящего среднего для обновления масштабирования весов модели.
        g111 = (gammag*g111+(1-gammag)*diffEw111**2)/ggt
        g112 = (gammag*g112+(1-gammag)*diffEw112**2)/ggt
        g113 = (gammag*g113+(1-gammag)*diffEw113**2)/ggt
        g114 = (gammag*g114+(1-gammag)*diffEw114**2)/ggt
        g121 = (gammag*g121+(1-gammag)*diffEw121**2)/ggt
        g122 = (gammag*g122+(1-gammag)*diffEw122**2)/ggt
        g123 = (gammag*g123+(1-gammag)*diffEw123**2)/ggt
        g124 = (gammag*g124+(1-gammag)*diffEw124**2)/ggt

        g211 = (gammag*g211+(1-gammag)*diffEw211**2)/ggt
        g212 = (gammag*g212+(1-gammag)*diffEw212**2)/ggt
        g221 = (gammag*g221+(1-gammag)*diffEw221**2)/ggt
        g222 = (gammag*g222+(1-gammag)*diffEw222**2)/ggt

        g311 = (gammag*g311+(1-gammag)*diffEw311**2)/ggt
        g312 = (gammag*g312+(1-gammag)*diffEw312**2)/ggt 

        # Корректируем веса
        # Веса нейронов входного слоя
        # Запишите формулу обновления весов модели с помощью инерции и масштабирования.
        w111 = w111 + h * v111 / np.sqrt(g111 + eps)
        w112 = w112 + h * v112 / np.sqrt(g112 + eps)
        w113 = w113 + h * v113 / np.sqrt(g113 + eps)
        w114 = w114 + h * v114 / np.sqrt(g114 + eps)

        w121 = w121 + h * v121 / np.sqrt(g121 + eps)
        w122 = w122 + h * v122 / np.sqrt(g122 + eps)
        w123 = w123 + h * v123 / np.sqrt(g123 + eps)
        w124 = w124 + h * v124 / np.sqrt(g124 + eps)

        # Веса нейронов скрытого слоя
        w211 = w211 + h * v211 / np.sqrt(g211 + eps)
        w212 = w212 + h * v212 / np.sqrt(g212 + eps)

        w221 = w221 + h * v221 / np.sqrt(g221 + eps)
        w222 = w222 + h * v222 / np.sqrt(g222 + eps)

        # Веса нейрона выходного слоя
        w311 = w311 + h * v311 / np.sqrt(g311 + eps)
        w312 = w312 + h * v312 / np.sqrt(g312 + eps)

        t += 1

    loss_new = loss_func(w111, w112, w113, w114, w121, w122, w123, w124,
                         w211, w212, w221, w222, w311, w312, dataY, dataX)
    if (abs(loss_new - loss_old) < eps):
        count_stop += 1
    else:
        count_stop = 0
    if loss_new < epsfunc:
        print(k + 1, 'я итерация алгоритма. Вычисляем значение функции потерь. E(w111,...,w312) = ',
              round(loss_new, 6))
        break
    if count_stop == 10:
        break
    else:
        print(k + 1, 'я итерация алгоритма. Вычисляем значение функции потерь. E(w111,...,w312) = ',
              round(loss_new, 6))

# Внесите значение loss_new на платформу
print(loss_new)

Задаем обучающую выборку
Функция сигмоида 1/(1+exp(-x))
1 этап алгоритма. Вычисляем значение функции потерь. E(w111,...,w312) =  2.051779
2 этап алгоритма. Вычисляем значение градиента функции потерь. diffE(w111,...,w314) = ( 3e-05 6.1e-05 9.1e-05 0.000121 4.5e-05 8.9e-05 0.000134 0.000179 -0.013375 -0.013639 -0.016046 -0.016362 1.046138 1.035333 )
3 этап алгоритма. Корректируем веса W = ( 0.010021 0.020042 0.030064 0.040085 0.050031 0.060063 0.070094 0.080125 -0.019363 -0.029547 -0.041232 -0.051453 0.682296 0.664733 )
1 я итерация алгоритма. Вычисляем значение функции потерь. E(w111,...,w312) =  1.155708
2 я итерация алгоритма. Вычисляем значение функции потерь. E(w111,...,w312) =  0.846165
3 я итерация алгоритма. Вычисляем значение функции потерь. E(w111,...,w312) =  0.892942
4 я итерация алгоритма. Вычисляем значение функции потерь. E(w111,...,w312) =  0.849581
5 я итерация алгоритма. Вычисляем значение функции потерь. E(w111,...,w312) =  1.280976
6 я итерация алгоритма. Вычисляем з

НОВЫЙ КОД